In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt

from numpy import loadtxt
from sklearn import cluster, datasets, metrics ,preprocessing
from sklearn.metrics import mean_absolute_error,accuracy_score,confusion_matrix
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from IPython.display import clear_output

def try_par(y_true, y_pred):
    minmae=99999
    Rnnn=0
    for nnn in range(800,1200):
        T1 = y_pred*nnn/1000
        mae = mean_absolute_error(y_true,T1)
        if minmae>mae:
            minmae=mae
            Rnnn=nnn
    return Rnnn/1000

def train_model(train_x,train_y,test_x,Re,num_class=3):
    params={'booster':'gbtree',
        'objective': 'multi:softmax', 
        'num_class':num_class,
        'gamma':0.05,
        'max_depth':7,
        'subsample':0.5,
        'colsample_bytree':0.6,
        'silent':1 ,
        'eta': 0.07,
        'seed':1000,
        'nthread':4
       }
    plst = list(params.items())
    num_rounds = 200
    offset = int(len(train_x)*0.8)
    train_x = train_x.reset_index(drop=True)
    train_y = train_y.reset_index(drop=True)
    xgtrain = xgb.DMatrix(train_x.loc[:offset,:], label=train_y[:offset+1])
    xgval = xgb.DMatrix(train_x.loc[offset:,:], label=train_y[offset:])
    watchlist = [(xgtrain, 'train'),(xgval, 'val')]
    model = xgb.train(plst, xgtrain, num_rounds, watchlist,early_stopping_rounds=40)

    test_x = test_x.reset_index(drop=True)
    Re = Re.reset_index(drop=True)
    pred1 = model.predict(xgb.DMatrix(Re),ntree_limit=model.best_iteration)
    pred2 = model.predict(xgb.DMatrix(test_x),ntree_limit=model.best_iteration)
    clear_output()
    return pred1,pred2

rgb_model = XGBRegressor(
                            learning_rate = 0.1,
                            n_estimators = 200,
                            max_depth= 7,
                            objective = 'reg:linear',
                            nthread = 4,
                            seed=1000
                         )
def train_model2(train_x,train_y,train_y2,test_x,Mask1,Mask2):
    rgb_model.fit(train_x[Mask1],train_y2[Mask1])
    T1,T2 = rgb_model.predict(train_x),rgb_model.predict(test_x)
    T1[T1<0]=0
    T2[T2<0]=0
    T3 = train_x['Premium_sum']*T1
    T4 = test_x['Premium_sum']*T2
    A = try_par(train_y[Mask2],T3[Mask2])
    T3,T4=T3*A,T4*A
    return T3,T4,A

def and2(A,B):
    C=A.copy()
    C[C]=B
    return C

def div1(Y,N1,N2):
    Z = Y.copy()
    A = Y.copy()
    Z[A>=N1] = 0
    Z[np.logical_and(A<N1,A>N2)] = 1
    Z[A<=N2] = 2
    return Z

In [2]:
train = pd.read_csv('../data/training_policy_c2.csv')
T = pd.read_csv('../data/duplicate_policy.csv',header=None)
T = T[0]
TF = train['Policy_Number']!=T[0]
for i in range(1,len(T)):
    TF = np.logical_and(TF,train['Policy_Number']!=T[i])
train = train[TF]
train = train.reset_index(drop=True)

test = pd.read_csv('../data/testing_policy_c2.csv')
train_y = train['Next_Premium'].copy()
train_y2 = (train_y/train['Premium_sum'])
test_PN = test['Policy_Number']

del train['Next_Premium']
del test['Next_Premium']
del train['Policy_Number']
del test['Policy_Number']

train_x = train
test_x = test

In [3]:
N1,N2= 0.9, 0
d00 = div1(train_y2,N1,N2)
P00,P01 = train_model(train_x,d00,test_x,train_x)

Mask000,Mask001,Mask002 = P00==0,P00==1,P00==2
Mask010,Mask011,Mask012 = P01==0,P01==1,P01==2

A00,A01=train_x['Premium_sum'] * 1,test_x['Premium_sum'] * 1
A00[Mask002]=0
A01[Mask012]=0
print(mean_absolute_error(train_y,A00))

2065.9400327689787


In [6]:
N1,N2= 1.02,0.95
d10 = div1(train_y2[Mask000],N1,N2)
P10,P11 = train_model(train_x[Mask000],d10,test_x[Mask010],train_x[Mask000])

Mask100,Mask101,Mask102 = and2(Mask000,P10==0),and2(Mask000,P10==1),and2(Mask000,P10==2)
Mask110,Mask111,Mask112 = and2(Mask010,P11==0),and2(Mask010,P11==1),and2(Mask010,P11==2)

Mask1 = np.logical_and(train_y2>0.5,train_y2<1.4)
Mask2 = np.logical_and(train_y2>0.7,train_y2<1.05)
Mask3 = np.logical_and(train_y2>0.7,train_y2<1.05)
T1,T2,A1 = train_model2(train_x,train_y,train_y2,test_x,Mask1,Mask100)
T3,T4,A2 = train_model2(train_x,train_y,train_y2,test_x,Mask2,Mask101)
T5,T6,A3 = train_model2(train_x,train_y,train_y2,test_x,Mask3,Mask102)

A10,A11 = A00.copy(),A01.copy()
A10[Mask100]=T1[Mask100]
A10[Mask101]=T3[Mask101]
A10[Mask102]=T5[Mask102]
A11[Mask110]=T2[Mask110]
A11[Mask111]=T4[Mask111]
A11[Mask112]=T6[Mask112]

print(A1,A2,A3,mean_absolute_error(train_y,A10))

1.023 1.005 0.99 1994.3464828726228


In [7]:
N1,N2= 0.85,0.7
d20 = div1(train_y2[Mask001],N1,N2)
P20,P21 = train_model(train_x[Mask001],d20,test_x[Mask011],train_x[Mask001])

Mask200,Mask201,Mask202 = and2(Mask001,P20==0),and2(Mask001,P20==1),and2(Mask001,P20==2)
Mask210,Mask211,Mask212 = and2(Mask011,P21==0),and2(Mask011,P21==1),and2(Mask011,P21==2)

Mask1 = np.logical_and(train_y2>0.6,train_y2<1.05)
Mask2 = np.logical_and(train_y2>0.55,train_y2<0.85)
Mask3 = np.logical_and(train_y2>0,train_y2<1.25)
T1,T2,A1 = train_model2(train_x,train_y,train_y2,test_x,Mask1,Mask200)
T3,T4,A2 = train_model2(train_x,train_y,train_y2,test_x,Mask2,Mask201)
T5,T6,A3 = train_model2(train_x,train_y,train_y2,test_x,Mask3,Mask202)

A20,A21 = A10.copy(),A11.copy()
A20[Mask200]=T1[Mask200]
A20[Mask201]=T3[Mask201]
A20[Mask202]=T5[Mask202]
A21[Mask210]=T2[Mask210]
A21[Mask211]=T4[Mask211]
A21[Mask212]=T6[Mask212]

print(A1,A2,A3,mean_absolute_error(train_y,A20))

1.015 1.012 0.889 1463.1011926509502


In [8]:
output_csv = pd.DataFrame({"Policy_Number":test_PN ,"Next_Premium":A21})
output_csv.to_csv("11.csv", sep=',', index=False, header=True, encoding='utf_8_sig')